# EQUIPE

- **Cristian Franklin Feitoza de Oliveira**
- **Harley Leite Silva**
- **George Anderson Batista Valente**
- **José Diogo Dutra Pacheco**

# Atualizações do Projeto

- **Layout foi reimaginado**
- **Função de cortes em vídeo finalizada**
- **Função de salvar vídeos cortados e editados finalizada**
- **Função de salvar frames finalizada**
- **Bugs referentes a cor e aplicação de filtros resolvidos**
- **Função de salvar imagens no histórico finalizada**
- **Função de aplicação de filtros independentes finalizada**
- **Bugs na abertura e reversão de vídeos resolvidos**
- **Aplicação de filtros em vídeos revertido implementada**

# image_handler

# Recursos

In [1]:
import cv2
import tkinter as tk
import numpy as np
from tkinter import filedialog, Toplevel, Canvas, Button, Frame
from tkinter import Toplevel, Label, Button, Canvas, filedialog
from PIL import Image, ImageTk
from tkinter import *
from PIL import Image, ImageTk
import shutil
import os
import customtkinter as ctk

# Filtros

In [2]:
k1 = np.array([[-1, -1, -1],[-1, 9, -1],[-1, -1, -1]])
k2 = np.array([[1, 0, 0],[0, 0, 0],[0, 0, -1]])
k3 = np.array([[0, 0, 0],[0, 1, 0],[0, 0, 0]])
k4 = np.array([[0, -1, 0],[-1, 5, -1],[0, -1, 0]])
k_blur = np.array([[1/9, 1/9, 1/9],[1/9, 1/9, 1/9],[1/9, 1/9, 1/9]], dtype=np.float32)
k_sharpen = np.array([[0, -1, 0],[-1, 5, -1],[0, -1, 0]], dtype=np.float32)
k_emboss = np.array([[2, 1, 0],[1, 0, -1],[0, -1, -2]], dtype=np.float32)
k_laplace = np.array([[0,1,0],[1,-4,1],[0,1,0]], dtype = np.float32)
k_sobel_x = np.array([[-1, 0, 1],[-2, 0, 2],[-1, 0, 1]], dtype=np.float32)
k_sobel_y = np.array([[1, 2, 1],[0, 0, 0],[-1, -2, -1]], dtype=np.float32)

# Variáveis Globais de Imagem

In [3]:
# Variáveis globais para gerenciar o estado da imagem
janela_principal = ctk.CTk()
imagem_original = None
imagem_atual = None
historico = []
selection = None  # Coordenadas da área selecionada
caminho = None
independente = False
canvas = None  # Definir canvas como variável global

# canvasController

In [4]:
# Funções para manipular a seleção no canvas
def iniciar_selecao(event):
    global selection
    selection = [event.x, event.y, event.x, event.y]  # Coordenadas iniciais

def fazer_selecao(event):
    global canvas, selection
    if selection:
        selection[2], selection[3] = event.x, event.y  # Atualizar coordenadas finais
        canvas.delete("selection")  # Remover seleção anterior
        canvas.create_rectangle(
            selection[0], selection[1], selection[2], selection[3], outline="red", tags="selection"
        )

def finalizar_selecao(event):
    global selection
    if selection:
        selection[2], selection[3] = event.x, event.y  # Coordenadas finais

# Função de Zoom

In [5]:
def aplicar_zoom(acao):
    global imagem_atual  # Declare que você está usando a variável global

    # Verifica se imagem_atual é None ou não é uma imagem válida
    if imagem_atual is None or not isinstance(imagem_atual, Image.Image):
        print("Nenhuma imagem carregada ou imagem inválida.")
        return

    # Agora você pode acessar a largura e altura
    largura, altura = imagem_atual.size

    if acao == "mais":
        # Lógica para aumentar o zoom
        nova_largura = int(largura * 1.1)  # Aumenta 10%
        nova_altura = int(altura * 1.1)
    elif acao == "menos":
        # Lógica para diminuir o zoom
        nova_largura = int(largura * 0.9)  # Diminui 10%
        nova_altura = int(altura * 0.9)
    else:
        print("Ação de zoom inválida.")
        return

    # Redimensiona a imagem
    imagem_atual = imagem_atual.resize((nova_largura, nova_altura), Image.LANCZOS)

    # Atualiza a visualização da imagem (supondo que você tenha um canvas ou algo similar)
    atualizar_canvas(imagem_atual)

def atualizar_canvas(imagem):
    
    imagem_tk = ImageTk.PhotoImage(imagem)
    canvas.create_image(0, 0, anchor='nw', image=imagem_tk)
    canvas.image = imagem_tk  # Mantém uma referência da imagem

# Função para Abrir Imagem

In [6]:
# Função para abrir a imagem
def abrir_imagem():
    global imagem_original, imagem_atual, caminho, canvas
    
    # Abrir a caixa de diálogo para selecionar uma imagem
    caminho_imagem = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])
    
    if caminho_imagem:
        caminho = caminho_imagem
        historico.clear()
        imagem_original = Image.open(caminho_imagem)
        imagem_atual = imagem_original
        
        # Converter a imagem para um formato que o Tkinter pode mostrar
        imagem_tk = ImageTk.PhotoImage(imagem_atual)
        
        # Exibir a imagem no canvas
        canvas.create_image(0, 0, anchor="nw", image=imagem_tk)
        canvas.image = imagem_tk  # Manter uma referência para a imagem

# Funções para aplicar filtro em imagens

In [7]:
def modo_def(modo):
    global imagem_atual, imagem_original  # Certifique-se de que imagem_original está definida
    if imagem_atual is None:
        print("Nenhuma imagem carregada. Por favor, abra uma imagem primeiro.")
        return None

    # Converter a imagem atual para um array NumPy para processamento com OpenCV
    imagem_cv = cv2.cvtColor(np.array(imagem_atual), cv2.COLOR_RGB2BGR)

    if modo == 'gray':
        return cv2.cvtColor(imagem_cv, cv2.COLOR_BGR2GRAY)
    elif modo == 'binary':
        gray = cv2.cvtColor(imagem_cv, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
        return binary
    elif modo == 'blur':
        imagem_rgb = cv2.cvtColor(imagem_cv, cv2.COLOR_BGR2RGB)
        return cv2.GaussianBlur(imagem_rgb, (5, 5), 0)
    elif modo == 'sharpen':
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        return cv2.filter2D(imagem_cv, -1, kernel)
    elif modo == 'emboss':
        kernel = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
        return cv2.filter2D(imagem_cv, -1, kernel)
    elif modo == 'laplace':
        imagem_rgb = cv2.cvtColor(imagem_cv, cv2.COLOR_BGR2RGB)
        laplacian = cv2.Laplacian(imagem_rgb, cv2.CV_64F)
        return cv2.convertScaleAbs(laplacian)  # Converte para uint8
    elif modo == 'sobel-x':
        sobel_x = cv2.Sobel(imagem_cv, cv2.CV_64F, 1, 0, ksize=5)
        return cv2.convertScaleAbs(sobel_x)  # Converte para uint8
    elif modo == 'sobel-y':
        sobel_y = cv2.Sobel(imagem_cv, cv2.CV_64F, 0, 1, ksize=5)
        return cv2.convertScaleAbs(sobel_y)  # Converte para uint8
    elif modo == 'edges':
        return cv2.Canny(imagem_cv, 100, 200)
    elif modo == 'original':
        if 'imagem_original' in globals():
            return np.array(imagem_original)  # Retorna a imagem original sem alterações
        else:
            print("Imagem original não está definida.")
            return None
    else:
        print("Modo não reconhecido.")
        return None

def aplicar_filtro(modo):
    global imagem_atual, independente
    
    if independente is True:
        imagem_atual=Image.fromarray(modo_def('original'))
    
    imagem_alterada = modo_def(modo)  # Supondo que modo_def retorne uma imagem alterada


    # Verifica se a imagem alterada é válida
    if imagem_alterada is None:
        print("A imagem alterada é None.")
        return
    
    # Verifica se a imagem alterada é um array NumPy
    if not isinstance(imagem_alterada, np.ndarray):
        print("A imagem alterada não é um array NumPy.")
        return

    # Adiciona ao histórico
    adicionar_historico()
    # Atualiza a imagem_atual com a imagem alterada
    imagem_atual = Image.fromarray(imagem_alterada)
    atualizar_canvas(imagem_atual)
    print("Filtro aplicado com sucesso.")
    
def voltar_cor():
    global imagem_atual, imagem_original
    if imagem_original is not None:
        imagem_atual = imagem_original
        imagem_tk = ImageTk.PhotoImage(imagem_atual)
        canvas.create_image(0, 0, anchor="nw", image=imagem_tk)
        canvas.image = imagem_tk  # Manter uma referência para a imagem

def mostrar_original():
    aplicar_filtro('original')

# Funções para a Janela Principal

In [8]:
def abrirJanelaPrincipal():
    global canvas, selection  # Tornar canvas acessível na função
    
    janela_principal.title("CapNow")
    janela_principal.geometry("1024x768")  # Tamanho inicial
    janela_principal.minsize(600, 600)  # Tamanho mínimo da janela
    janela_principal.resizable(True, True)
    ctk.set_appearance_mode("light")

    # Limpar histórico e seleção
    selection = None

    # Configurar o frame principal
    frame = ctk.CTkFrame(janela_principal)
    frame.grid(row=0,column=0, padx=20, pady=20, sticky="nsew")

    janela_principal.grid_rowconfigure(0, weight=1)
    janela_principal.grid_columnconfigure(0, weight = 1)
    #frame.grid_rowconfigure(1,weight=1)
    #frame.grid_columnconfigure(1,weight=2)
    #frame.grid_columnconfigure(0,weight=1)

    # Configurando o label da página principal
    label = ctk.CTkLabel(frame, text='Ferramenta de Manipulação de Imagens e Vídeos', text_color = "black", fg_color = "whitesmoke", font=('semibold',20), corner_radius=5)
    #label = tk.Label(frame, text="Ferramenta de Manipulação de Imagens e Vídeos", font=("Arial", 18), fg="black", bg="lightgray")

    # Canvas para a imagem
    canvas = Canvas(frame, width=300, height=300, bg="#242424")

    #CheckButton para cascata
    cascata_check = tk.BooleanVar()
    #check = tk.Checkbutton(frame, text="Ativar Filtro em Cascata", variable=cascata_check)

    # Ativar seleção no canvas
    canvas.bind("<ButtonPress-1>", iniciar_selecao)
    canvas.bind("<B1-Motion>", fazer_selecao)
    canvas.bind("<ButtonRelease-1>", finalizar_selecao)

    # Adicionando nas posições desejadas
    # Row 0
    frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
    label.grid(row=0, column=0, columnspan=10, padx=10, pady=5, sticky="ew")

    # Row 1
    button = ctk.CTkButton(frame, text="Abrir imagem",font=("bold", 16),command=abrir_imagem, fg_color="#1EA5BA")
    button.grid(row = 1, column=0, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Abrir Imagem", command=abrir_imagem).grid(row=1, column=0, padx=10, pady=5, sticky="ew")
    canvas.grid(row=2, column=1, columnspan=10, rowspan=10, sticky="nsew", pady=10)

    # Row 2
    button = ctk.CTkButton(frame, text="Abrir vídeo",font=("bold", 16),command=abrir_video, fg_color="#1EA5BA")
    button.grid(row = 1, column=1, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Abrir vídeo", command=abrir_video).grid(row=2, column=0, padx=10, pady=5, sticky="ew")

    # Row 3
    #check = ctk.CTkCheckBox(frame, text = "Ativar filtro em cascata")
    #check.grid(row=3, column = 0, padx = 5, pady=10)
    #check.grid(row=3, column=0, padx=5,pady=10)

    ## Menu de Filtros de Convolução
   # Adicionando botões de filtros
    adicionar_botoes_filtros(frame)

    ## Menu de Modo de Cor
    # Row 11
    modeCorLabel = ctk.CTkLabel(frame, text='Modo de Cor', text_color = "white", fg_color = "#3A4046",  corner_radius=5)
    #modeCorLabel = tk.Label(frame, text="Modo de Cor")
    modeCorLabel.grid(row=12, column=0, padx=5,pady=10)
    # Row 12
    ctk.CTkButton(frame, text="Cinza", command=lambda: aplicar_filtro('gray')).grid(row=13, column=0, padx=10, pady=5, sticky="ew")
    # Row 13
    ctk.CTkButton(frame, text="Binário", command=lambda: aplicar_filtro('binary')).grid(row=14, column=0, padx=10, pady=5, sticky="ew")
    # Row 14
    ctk.CTkButton(frame, text="Cor", command=voltar_cor).grid(row=15, column=0, padx=10, pady=5, sticky="ew")

    ## Controles Gerais
    # Row 11
    menuGeralLabel = ctk.CTkLabel(frame, text='Controles Gerais', text_color = "white", fg_color = "#3A4046",  corner_radius=5)
    menuGeralLabel.grid(row=12, column=1, padx=5,pady=10)
    # Row 12
    ctk.CTkButton(frame, text="Desfazer", command=desfazer).grid(row=13, column=1, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Desfazer", command=desfazer).grid(row=12, column=1, padx=10, pady=5, sticky="ew")
    # Row 13
    ctk.CTkButton(frame, text="Original", command=mostrar_original).grid(row=14, column=1, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Original", command=mostrar_original).grid(row=13, column=1, padx=10, pady=5, sticky="ew")
    # Row 14
    ctk.CTkButton(frame, text="Salvar Imagem", command=salvar_imagem, fg_color = "#34C169").grid(row=15, column=1, padx=10, pady=5, sticky="ew")
    ctk.CTkButton(frame, text="Salvar Imagens no Histórico", command=salvar_imagens_historico, fg_color = "#34C169").grid(row=16, column=1, padx=10, pady=5, sticky="ew")
    ctk.CTkButton(frame, text="Ativar/Desativar Filtro independente", command=filtro_independente).grid(row=17, column=1, padx=10, pady=5, sticky="ew")
#Button(frame, text="Salvar Imagem", command=salvar_imagem).grid(row=14, column=1, padx=10, pady=5, sticky="ew")
    # Row 15
    #Button(frame, text="Fechar").grid(row=15, column=1, padx=10, pady=5, stick="ew")

    ## Opções do Quadro
    # Row 11
    menuQuadroLabel = ctk.CTkLabel(frame, text='Opções Quadro', text_color = "white", fg_color = "#3A4046",  corner_radius=5)
    menuQuadroLabel.grid(row=12, column=2, padx=5,pady=10)
    # Row 12
    ctk.CTkButton(frame, text="Zoom na Seleção", command=aplicar_zoom_selecao).grid(row=13, column=2, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Zoom na Seleção", command=aplicar_zoom_selecao).grid(row=12, column=2, padx=10, pady=5, sticky="ew")
    # Row 13
    ctk.CTkButton(frame, text="Zoom +", command=lambda: aplicar_zoom("mais")).grid(row=14, column=2, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Zoom +", command=lambda: aplicar_zoom("mais")).grid(row=13, column=2, padx=10, pady=5, sticky="ew")
    # Row 14
    ctk.CTkButton(frame, text="Zoom -", command=lambda: aplicar_zoom("menos")).grid(row=15, column=2, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Zoom -", command=lambda: aplicar_zoom("menos")).grid(row=14, column=2, padx=10, pady=5, sticky="ew")

     ## Opções de Vídeo
    # Row 11
    menuVideoLabel = ctk.CTkLabel(frame, text='Opções Vídeo', text_color = "white", fg_color = "#3A4046",  corner_radius=5)
    menuVideoLabel.grid(row=12, column=3, columnspan=2, padx=5,pady=10)
    # Row 12
    ctk.CTkButton(frame, text="Reproduzir").grid(row=13, column=3, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Reproduzir").grid(row=12, column=3, padx=10, pady=5, stick="ew")
    ctk.CTkButton(frame, text="Reverter").grid(row=13, column=4, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Reverter").grid(row=12, column=4, padx=5, pady=5, stick="ew")
    
    # Row 13
    ctk.CTkButton(frame, text="Acelerar").grid(row=14, column=3, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Acelerar").grid(row=13, column=3, padx=10, pady=5, stick="ew")
    ctk.CTkButton(frame, text="Desacelerar").grid(row=14, column=4, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Desacelerar").grid(row=13, column=4, padx=5, pady=5, stick="ew")
    
    # Row 14
    ctk.CTkButton(frame, text="Cortar").grid(row=15, column=3, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Cortar").grid(row=14, column=3, padx=10, pady=5, stick="ew")
    ctk.CTkButton(frame, text="Salvar Frame").grid(row=15, column=4, padx=10, pady=5, sticky="ew")
    #Button(frame, text="Salvar Frame").grid(row=14, column=4, padx=5, pady=5, stick="ew")

    # Inicia a interface
    return janela_principal

def adicionar_botoes_filtros(frame):
    # Filtros de Convolução
    Button(frame, text="Blur", command=lambda: aplicar_filtro('blur')).grid(row=5, column=0, padx=10, pady=5, sticky="ew")
    Button(frame, text="Sharpen", command=lambda: aplicar_filtro('sharpen')).grid(row=6, column=0, padx=10, pady=5, sticky="ew")
    Button(frame, text="Emboss", command=lambda: aplicar_filtro('emboss')).grid(row=7, column=0, padx=10, pady=5, sticky="ew")
    Button(frame, text="Laplacian", command=lambda: aplicar_filtro('laplace')).grid(row=8, column=0, padx=10, pady=5, sticky="ew")
    Button(frame, text="Sobel X", command=lambda: aplicar_filtro('sobel-x')).grid(row=9, column=0, padx=10, pady=5, sticky="ew")
    Button(frame, text="Sobel Y", command=lambda: aplicar_filtro('sobel-y')).grid(row=10, column=0, padx=10, pady=5, sticky="ew")
    Button(frame, text="Canny", command=lambda: aplicar_filtro('edges')).grid(row=11, column=0, padx=10, pady=5, sticky="ew")


In [9]:
#Button(frameControles, text="Salvar Imagens no Histórico", command=salvar_imagens_historico).grid(row=1, column=2, padx=10, pady=5, sticky="ew")
#Button(frameControles, text="Ativar/Desativar Filtro independente", command=filtro_independente).grid(row=1, column=1, padx=10, pady=5, sticky="ew")

# Funções de Manipulação de Imagem

In [10]:
def filtro_independente():
    global imagem_atual, imagem_original, independente
    if independente is False:
        print("Filtro Independente Ativado")
        independente = not independente
        adicionar_historico()
        mostrar_original()
    else:
        print("Filtro Independente Desativado")
        independente = not independente
        desfazer()
        
# Função para adicionar histórico
def adicionar_historico():
    global imagem_atual, historico, independente
    # Verifica se imagem_atual é None
    if imagem_atual is None:
        print("Nenhuma imagem para adicionar ao histórico.")
        return
    
    # Adiciona a imagem atual ao histórico
    if independente is True:
        print("Filtro Independente ativado, impossível salvar no histórico.")
    else:
        historico.append(imagem_atual.copy())  # Usar copy() para evitar referências
        print("Imagem adicionada ao histórico.")
    

# Função para desfazer
def desfazer():
    global imagem_atual, historico, canvas, independente, imagem_original

    if not historico:
        print("Nenhuma ação para desfazer.")
        imagem_atual=imagem_original
        imagem_cv = np.array(imagem_atual)
        imagem_cv=normalizar_frame(imagem_cv)
        imagem_cv = Image.fromarray(imagem_cv)
        atualizar_canvas(imagem_cv)
        return
    elif independente is True:
        print("Filtro Independente Ativado. Imposssível realizar operação.")
    imagem_atual = historico.pop()  # Remove a última imagem do histórico
    imagem_cv = np.array(imagem_atual)
    imagem_cv=normalizar_frame(imagem_cv)
    imagem_cv = Image.fromarray(imagem_cv)
    atualizar_canvas(imagem_cv)

# Função para aplicar zoom na área selecionada, mantendo proporções
def aplicar_zoom_selecao():
    global imagem_atual, selection
    if imagem_atual is None:
        print("Nenhuma imagem carregada para aplicar zoom.")
        return

    if not selection:
        print("Nenhuma área selecionada para aplicar zoom.")
        return
        
    
    adicionar_historico()
    x1, y1, x2, y2 = selection

    # Certificar-se de que as coordenadas estão dentro dos limites
    largura, altura = imagem_atual.size
    x1, x2 = max(0, min(x1, x2)), min(largura, max(x1, x2))
    y1, y2 = max(0, min(y1, y2)), min(altura, max(y1, y2))

    # Recortar a área selecionada
    area_selecionada = imagem_atual.crop((x1, y1, x2, y2))

    # Calcular nova dimensão preservando proporção
    largura_selecao, altura_selecao = area_selecionada.size
    fator_zoom = 2  # Fator de zoom
    nova_largura = int(largura_selecao * fator_zoom)
    nova_altura = int(altura_selecao * fator_zoom)

    area_ampliada = area_selecionada.resize((nova_largura, nova_altura), Image.Resampling.LANCZOS)

    # Centralizar no Canvas mantendo proporção
    canvas_largura, canvas_altura = 500, 500
    proporcao_area = nova_largura / nova_altura
    proporcao_canvas = canvas_largura / canvas_altura

    if proporcao_area > proporcao_canvas:
        # Ajustar pela largura
        largura_final = canvas_largura
        altura_final = int(largura_final / proporcao_area)
    else:
        # Ajustar pela altura
        altura_final = canvas_altura
        largura_final = int(altura_final * proporcao_area)

    area_final = area_ampliada.resize((largura_final, altura_final), Image.Resampling.LANCZOS)

    # Criar uma nova imagem com fundo cinza e centralizar a área ampliada
    imagem_zoom = Image.new("RGB", (canvas_largura, canvas_altura), "gray")
    offset_x = (canvas_largura - largura_final) // 2
    offset_y = (canvas_altura - altura_final) // 2
    imagem_zoom.paste(area_final, (offset_x, offset_y))

    imagem_atual = imagem_zoom
    atualizar_canvas(imagem_atual)

def salvar_imagens_historico():
    global historico
    if not historico:
        print("Nenhuma imagem para salvar.")
        return
    
    # Solicita ao usuário um diretório para salvar as imagens
    diretorio = filedialog.askdirectory()
    if not diretorio:
        print("Nenhum diretório selecionado.")
        return

    # Salva cada imagem do histórico em um arquivo separado
    for i, h in enumerate(historico):
        # Cria um nome de arquivo único para cada imagem
        nome_arquivo = os.path.join(diretorio, f"imagem_historico_{i + 1}.png")
        h.save(nome_arquivo)
        print(f"Imagem salva em: {nome_arquivo}")
        
def salvar_imagem():
    global imagem_atual
    if imagem_atual is None:
        print("Nenhuma imagem carregada para salvar.")
        return

    caminho = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG", "*.png"), ("JPEG", "*.jpg")])
    if caminho:
        imagem_atual.save(caminho)
        print(f"Imagem salva em: {caminho}")

# Variáveis Globais de Vídeo

In [11]:
# Variáveis globais para controle de vídeo
video_capture = None
video_capture_webcam = None
canvas_video = None
frame_atual = None
tempo_atual = 10  # Tempo em milissegundos entre os quadros
filtro_atual = None
paused = False
tempos_corte = [0]  # Lista para armazenar os tempos de corte
tempo_total = 0  # Tempo total do vídeo
video_path = ""# Caminho do vídeo
cont_frames = 0 #contador de frames
cont_frames_webcam = 0 #contador de frames da webcam
parador = False
frames_webcam=[]
webcam_used = False
janelas_abertas = []
frames_reverso = []
video_status = False

# Funções para Criar Janela de Vídeo

In [12]:
def criar_janela_video(title):
    global canvas_video, video_capture, frame_video

    # Criar janela de vídeo
    janela_video = Toplevel()
    janela_video.title(title)

    # Criar um Frame para o Canvas e a barra de rolagem
    frame_canvas = Frame(janela_video)
    frame_canvas.grid(row=0, column=0)

    # Criar Canvas com as dimensões do vídeo
    canvas_video = Canvas(frame_canvas, width=700, height=600, bg="black")
    canvas_video.grid(row=0, column=0, sticky="nsew")

    # Criar barra de rolagem
    scrollbar = Scrollbar(frame_canvas, orient=VERTICAL, command=canvas_video.yview)
    scrollbar.grid(row=0, column=1, sticky='ns')

    # Configurar o Canvas para usar a barra de rolagem
    canvas_video.configure(yscrollcommand=scrollbar.set)

    # Criar um Frame dentro do Canvas
    frame_video = Frame(canvas_video)
    canvas_video.create_window((0, 0), window=frame_video, anchor='nw')

    # Configurar o Frame para expandir
    frame_video.bind("<Configure>", lambda e: canvas_video.configure(scrollregion=canvas_video.bbox("all")))

    # Criar um Frame para os botões
    frame_botoes = Frame(janela_video)
    frame_botoes.grid(row=0, column=1, padx=10, pady=10)

    # Adicionar botões de controle de reprodução
    Button(frame_botoes, text="Reproduzir", command=reproduzir_video).grid(row=0, column=0, padx=5, pady=5)
    Button(frame_botoes, text="Pausar", command=pause_video).grid(row=0, column=1, padx=5, pady=5)
    Button(frame_botoes, text="Reproduzir Vídeo em Reverso", command=reproduzir_video_reverso_if).grid(row=0, column=2, padx=5, pady=5)
    Button(frame_botoes, text="Capturar Quadro", command=capturar_quadro).grid(row=0, column=3, padx=5, pady=5)
    Button(frame_botoes, text="Fechar", command=lambda: fechar_video(janela_video)).grid(row=0, column=4, padx=5, pady=5)
    Button(frame_botoes, text="Registrar Tempo", command=registrar_tempo).grid(row=5, column=0, padx=5, pady=5)
    Button(frame_botoes, text="Gerar Vídeos", command=gerar_videos).grid(row=5, column=1, padx=5, pady=5)
    Button(frame_botoes, text="Salvar Frames", command=salvar_frames).grid(row=5, column=2, padx=5, pady=5)

    # Botão para usar a webcam
    Button(frame_botoes, text="Usar Webcam", command=usar_webcam).grid(row=1, column=0, columnspan=5, padx=5, pady=5)

    # Adicionar botões de filtro na linha 2
    adicionar_botoes_filtros_video(frame_botoes)

    # Botões para aumentar e diminuir o tempo na linha 3
    Button(frame_botoes, text="Aumentar Tempo", command=aumentar_tempo).grid(row=4, column=0, padx=5, pady=5)
    Button(frame_botoes, text="Diminuir Tempo", command=diminuir_tempo).grid(row=4, column=1, padx=5, pady=5)

def adicionar_botoes_filtros_video(frame):
    # Adiciona botões para aplicar filtros usando grid
    Button(frame, text="Cinza", command=lambda: mudar_filtro('gray')).grid(row=2, column=0, padx=5, pady=5)
    Button(frame, text="Binário", command=lambda: mudar_filtro('binary')).grid(row=2, column=1, padx=5, pady=5)
    Button(frame, text="Blur", command=lambda: mudar_filtro('blur')).grid(row=2, column=2, padx=5, pady=5)
    Button(frame, text="Sharpen", command=lambda: mudar_filtro('sharpen')).grid(row=2, column=3, padx=5, pady=5)
    Button(frame, text="Emboss", command=lambda: mudar_filtro('emboss')).grid(row=3, column=0, padx=5, pady=5)
    Button(frame, text="Laplacian", command=lambda: mudar_filtro('laplace')).grid(row=3, column=1, padx=5, pady=5)
    Button(frame, text="Sobel X", command=lambda: mudar_filtro('sobel-x')).grid(row=3, column=2, padx=5, pady=5)
    Button(frame, text="Sobel Y", command=lambda: mudar_filtro('sobel-y')).grid(row=3, column=3, padx=5, pady=5)
    Button(frame, text="Canny", command=lambda: mudar_filtro('edges')).grid(row=4, column=2, padx=5, pady=5)
    Button(frame, text="Sem Filtro", command=lambda: mudar_filtro('original')).grid(row=4, column=3, padx=5, pady=5)

# Funções para Reprodução de Vídeo

In [13]:
def reproduzir_video():
    global video_capture, canvas_video, frame_atual, filtro_atual, paused, tempo_total, cont_frames, video_path, video_status
    if paused: # Se estiver pausado, não faz nada
        return
    
    if not video_capture or not video_capture.isOpened():
        print("Nenhum vídeo carregado para reprodução.")
        abrir_video()
        return

    if video_status is True:
        video_status = not video_status
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        return reproduzir_video()
        
    # Ler próximo quadro do vídeo
    ret, frame = video_capture.read()
    cont_frames+=1
    if not ret:
        video_status = not video_status
        print("Fim do vídeo ou erro ao ler o quadro. Reiniciando o vídeo.")
        return
        #video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reinicia o vídeo
        #return reproduzir_video()  # Tenta reproduzir novamente
    if not paused:
        # Aplicar filtro se um filtro atual estiver definido
        frame_processado=None
        if filtro_atual:
            frame = modo_def_video(frame, filtro_atual)
        frame_processado = normalizar_frame(frame)
        frame_processado = cv2.cvtColor(frame_processado, cv2.COLOR_BGR2RGB)
        frame_atual = ImageTk.PhotoImage(Image.fromarray(frame_processado))
    
        # Atualizar o Canvas
        canvas_video.create_image(0, 0, anchor="nw", image=frame_atual)
        canvas_video.img = frame_atual  # Previne garbage collection
    
        # Continuar reprodução
        canvas_video.after(tempo_atual, reproduzir_video)

def reproduzir_video_webcam():
    global video_capture_webcam, canvas_video, frame_atual, filtro_atual, paused, tempo_total, cont_frames_webcam, frames_webcam, video_status
    if paused: # Se estiver pausado, não faz nada
        tempo_total=cont_frames_webcam;
        return
    if not video_capture_webcam or not video_capture_webcam.isOpened():
        print("Nenhum vídeo carregado para reprodução.")
        return

  
    # Ler próximo quadro do vídeo
    ret, frame = video_capture_webcam.read()
    cont_frames_webcam+=1
    if not ret:
        print("Fim do vídeo ou erro ao ler o quadro. Reiniciando o vídeo.")
        video_capture_webcam.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reinicia o vídeo
        return reproduzir_video_webcam()  # Tenta reproduzir novamente
    if not paused:
        frame_processado=None
        # Aplicar filtro se um filtro atual estiver definido
        if filtro_atual:
            frame = modo_def_video(frame, filtro_atual)
        frame_processado = normalizar_frame(frame)
        frame_processado = cv2.cvtColor(frame_processado, cv2.COLOR_BGR2RGB)
        frame_atual = ImageTk.PhotoImage(Image.fromarray(frame_processado))
    
        # Atualizar o Canvas
        canvas_video.create_image(0, 0, anchor="nw", image=frame_atual)
        canvas_video.img = frame_atual  # Previne garbage collection
    
        # Continuar reprodução
        canvas_video.after(tempo_atual, reproduzir_video_webcam)

# Funções de seleção de Vídeo

In [14]:
def abrir_video():
    global video_capture, canvas_video, tempo_total, video_path, video_capture_reverso

    # Seleção de arquivo de vídeo
    caminho = filedialog.askopenfilename(filetypes=[("Vídeos", "*.mp4;*.avi;*.mov")])
    if not caminho:
        return

    # Abrir vídeo com OpenCV
    video_capture = cv2.VideoCapture(caminho)
    if not video_capture.isOpened():
        print("Erro ao abrir o vídeo.")
        return
    # Armazenar o caminho do vídeo
    video_path = caminho
    print("video_path"+video_path)

    # Calcular o tempo total do vídeo
    tempo_total = contar_frames()
    print(f"Duração total do vídeo: {tempo_total} frames")
    # Criar janela de vídeo
    janelas_abertas.append("Vizualizar Video")
    criar_janela_video("Vizualizar Video")
def usar_webcam():
    global video_capture_webcam, webcam_used
    webcam_used=True
    video_capture_webcam = cv2.VideoCapture(0)  # 0 é o índice da webcam padrão
    if not video_capture_webcam.isOpened():
        print("Erro ao acessar a webcam.")
        return
    # Criar janela de vídeo
    janelas_abertas.append("Webcam")
    fechar_janela_por_titulo("Vizualizar Video")
    criar_janela_video("Webcam")
    # Inicia a reprodução da webcam
    reproduzir_video_webcam()

# Funções para manipulação de Vídeo

In [15]:
def normalizar_frame(frame):
    if frame is None:
        return None
    if len(frame.shape) == 2:  # Se for uma imagem em escala de cinza
        return cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)  # Converte para BGR
    return frame  # Retorna o frame como está se já for BGR
def aumentar_tempo():
    global tempo_atual
    tempo_atual += 10  # Aumenta o tempo em 50 ms
    print(f"Tempo atual aumentado: {tempo_atual} ms")

def diminuir_tempo():
    global tempo_atual
    tempo_atual = max(0, tempo_atual - 10)  # Diminui o tempo em 50 ms, não permitindo valores negativos
    print(f"Tempo atual diminuído: {tempo_atual} ms")
def pause_video():
    global paused
    paused = not paused  # Alterna o estado de pausa
    if paused:
        print("Vídeo pausado.")
    else:
        print("Vídeo retomado.")
        reproduzir_video()
def capturar_quadro():
    global frame_atual

    if not frame_atual:
        print("Nenhum quadro disponível para capturar.")
        return

    # Salvar o quadro atual como imagem
    frame_atual._PhotoImage__photo.write("quadro_capturado.png", format="png")
    print("Quadro capturado e salvo como 'quadro_capturado.png'.")

def fechar_video(janela):
    global video_capture
    if video_capture_webcam  and video_capture_webcam.isOpened():
        video_capture_webcam.release()
        fechar_janela_por_titulo("Webcam")
    elif video_capture and video_capture.isOpened():
        video_capture.release()
        fechar_janela_por_titulo("Vizualizar Video")
        fechar_janela_por_titulo("Video Reverso")

def fechar_janela(titulo, janela):
    # Remove o título da janela da lista e fecha a janela
    if titulo in janelas_abertas:
        janelas_abertas.remove(titulo)
    janela.destroy()

def fechar_janela_por_titulo(titulo):
    # Fecha a janela com o título especificado
    global janela_principal
    for janela in janela_principal.winfo_children():
        if isinstance(janela, tk.Toplevel) and janela.title() == titulo:
            fechar_janela(titulo, janela)
            break

def contar_frames():
    global video_path
    cap_cont = cv2.VideoCapture(video_path)
    
    if not cap_cont.isOpened():
        print("Erro: O vídeo não está aberto.")
        return 0

    total_frames = 0

    while True:
        ret, frame = cap_cont.read()
        if not ret:
            break  # Sai do loop se não houver mais frames

        total_frames += 1

    
    cap_cont.release()

    return total_frames

# Funções para aplicar filtros em Vídeo

In [16]:
def modo_def_video(frame, modo):
    if frame is None:
        print("Nenhum quadro carregado. Por favor, abra um vídeo primeiro.")
        return None

    if modo == 'gray':
        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif modo == 'binary':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
        return binary
    elif modo == 'blur':
        return cv2.GaussianBlur(frame, (5, 5), 0)
    elif modo == 'sharpen':
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        return cv2.filter2D(frame, -1, kernel)
    elif modo == 'emboss':
        kernel = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
        return cv2.filter2D(frame, -1, kernel)
    elif modo == 'laplace':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Converte para escala de cinza
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        return cv2.convertScaleAbs(laplacian)  # Converte para uint8
    elif modo == 'sobel-x':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Converte para escala de cinza
        sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        return cv2.convertScaleAbs(sobel_x)  # Converte para uint8
    elif modo == 'sobel-y':
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Converte para escala de cinza
        sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        return cv2.convertScaleAbs(sobel_y)  # Converte para uint8
    elif modo == 'edges':
        return cv2.Canny(frame, 100, 200)
    elif modo == 'original':
        return frame
    else:
        print("Modo não reconhecido.")
        return None

def mudar_filtro(novo_filtro):
    global filtro_atual
    filtro_atual = novo_filtro  # Atualiza o filtro atual

# Funções para reverter vídeo

In [17]:
def reproduzir_video_reverso_if():
    global frames_reverso
    if not frames_reverso:
        reproduzir_video_reverso() 
    else:
        iniciar_reproducao()
def reproduzir_video_reverso():
    global video_capture, canvas_video, paused, frames_reverso
    if paused:
        return
    cap_ = video_capture
    
    # Ler todos os frames do vídeo e armazená-los
    while True:
        ret, frame = cap_.read()
        if not ret:
            break
        frames_reverso.append(frame)

    # Verifica se há frames para reproduzir
    if not frames_reverso:
        print("Nenhum frame foi lido do vídeo.")
        cap_.release()  # Libera o objeto de captura antes de retornar
        return

    print(f"Total de frames lidos: {len(frames_reverso)}")  # Para verificar quantos frames foram lidos

    janelas_abertas.append("Video Reverso")
    fechar_janela_por_titulo("Vizualizar Video")
    criar_janela_video("Video Reverso")
    

    # Função interna para reproduzir os frames em ordem reversa
def reproduzir_frames_reverso(index):
    global frame_atual, filtro_atual, paused, frames_reverso
    frame_processado=None
    if not paused:
        if index < 0:
            index = len(frames_reverso) - 1  # Reinicia a reprodução
        frame = frames_reverso[index]
        if filtro_atual:
            frame = modo_def_video(frame, filtro_atual)
        frame_processado=normalizar_frame(frame)
        frame_processado= cv2.cvtColor(frame_processado, cv2.COLOR_BGR2RGB)
        frame_atual = ImageTk.PhotoImage(Image.fromarray(frame_rgb))
    
        # Atualiza o Canvas
        canvas_video.create_image(0, 0, anchor="nw", image=frame_atual)
        canvas_video.img = frame_atual  # Previne garbage collection
    
        # Continuar reprodução
        canvas_video.after(tempo_atual, reproduzir_frames_reverso, index - 1)
    
    # Função para iniciar a reprodução
def iniciar_reproducao():
    global frames_reverso
    reproduzir_frames_reverso(len(frames_reverso) - 1)

# Função para cortar vídeo

In [18]:
def registrar_tempo(event=None):
    global video_capture, tempos_corte, cont_frames, cont_frames_webcam

    # Obter o tempo atual do vídeo
    print(cont_frames_webcam)
    print(webcam_used)
    if webcam_used:
        tempo_atual = cont_frames_webcam
    else:
        tempo_atual = cont_frames
    print(tempo_atual)
    # Verifique se o tempo atual já está na lista para evitar duplicatas
    if not tempos_corte or tempos_corte[-1] != tempo_atual:
        tempos_corte.append(tempo_atual)
        print(f"Tempo registrado: {tempo_atual} frame")
    else:
        print(f"Tempo {tempo_atual} segundos já registrado, não adicionando novamente.")

def extrair_intervalo_frames(inicio, fim):
    global frames_webcam
    # Verifica se os índices estão dentro dos limites da lista
    if inicio < 0 or fim >= len(frames_webcam) or inicio > fim:
        print("Índices inválidos. Certifique-se de que 'inicio' e 'fim' estão dentro dos limites da lista de frames.")
        return []

    # Extrai e retorna o intervalo de frames
    return frames_webcam[inicio:fim + 1]


def mapear_frames(inicio, fim):
    global video_path, filtro_atual

    cap = cv2.VideoCapture(video_path)
    # Verifica se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return
    
    # Lista para armazenar os frames
    frames = []
    
    # Define o frame inicial
    cap.set(cv2.CAP_PROP_POS_FRAMES, inicio)
    
    # Lê os frames entre os pontos de corte
    for frame_index in range(inicio, fim + 1):
        ret, frame = cap.read()
        if filtro_atual:
            frame = modo_def_video(frame, filtro_atual)
        frame = normalizar_frame(frame)
        if not ret:
            print(f"Não foi possível ler o frame {frame_index}.")
            break
        frames.append(frame)
    
    # Libera o objeto de captura
    cap.release()
    
    return frames

def gerar_video(frames, output_path, fps=30):
    if not frames:
        print("A lista de frames está vazia.")
        return
    print(frames[0].shape)
    # Obtém as dimensões do primeiro frame
    height, width, _ = frames[0].shape
    
    # Define o codec e cria o objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Escreve cada frame no vídeo
    for frame in frames:
        out.write(frame)
    
    # Libera o objeto VideoWriter
    out.release()
    print(f"Vídeo salvo em: {output_path}")

def gerar_videos():
    global tempos_corte, tempo_total, video_path, webcam_used
    
    if webcam_used:
        tempo_total=cont_frames_webcam;
        
    frames_mapeados = []

    tempos_corte.append(tempo_total-1)

    # Verifique os tempos de corte
    print(f"Tempos de corte registrados: {tempos_corte}")

    # Gerar vídeos com base nos tempos de corte
    for i in range(len(tempos_corte) - 1):
        start_time = tempos_corte[i]
        end_time = tempos_corte[i + 1]
        if(webcam_used):
            frames_mapeados.append(extrair_intervalo_frames(start_time, end_time))
        else:
            frames_mapeados.append(mapear_frames(start_time, end_time))

        # Gera um vídeo para cada sublista de frames
    for i, frames in enumerate(frames_mapeados):
        output_path = f'video_{i}.mp4'  # Define o nome do arquivo de saída
        gerar_video(frames, output_path)

    print("Todos os vídeos foram gerados.")

def salvar_frames():
    global tempos_corte, webcam_used, tempo_total
    if webcam_used:
        tempo_total=cont_frames_webcam;
    tempos_corte.append(tempo_total-1)
    # Itera sobre os tempos de corte para salvar os frames
    for i in range(len(tempos_corte) - 1):
        start_time = tempos_corte[i]
        end_time = tempos_corte[i + 1]
        
        # Define o nome da pasta para o conjunto atual
        pasta_destino = f'frames_salvos_{i + 1}'
        
        # Se a pasta já existir, remove-a para sobrescrever
        if os.path.exists(pasta_destino):
            shutil.rmtree(pasta_destino)  # Remove a pasta e seu conteúdo
        
        # Cria a nova pasta de destino
        os.makedirs(pasta_destino)
        print(f"Pasta '{pasta_destino}' criada ou sobrescrita.")
        
        if webcam_used:
            frames = extrair_intervalo_frames(start_time, end_time)
        else:
            frames = mapear_frames(start_time, end_time)
        
        # Salva cada frame na pasta de destino
        for j, frame in enumerate(frames):
            if frame is not None:  # Verifica se o frame não é None
                # Define o nome do arquivo
                nome_arquivo = os.path.join(pasta_destino, f'frame_{start_time + j}.png')
                # Salva o frame como uma imagem
                cv2.imwrite(nome_arquivo, frame)
                print(f'Salvo: {nome_arquivo}')
            else:
                print(f'Frame {start_time + j} é None e não será salvo.')

# Função Main

In [19]:
def main():
    abrirJanelaPrincipal().mainloop()
main()

video_pathC:/Users/PC/Documents/GitHub/capNow/teti_imagens/caleido.mp4
Duração total do vídeo: 1199 frames
0
False
47
Tempo registrado: 47 frame
0
False
111
Tempo registrado: 111 frame
Pasta 'frames_salvos_1' criada ou sobrescrita.
Salvo: frames_salvos_1\frame_0.png
Salvo: frames_salvos_1\frame_1.png
Salvo: frames_salvos_1\frame_2.png
Salvo: frames_salvos_1\frame_3.png
Salvo: frames_salvos_1\frame_4.png
Salvo: frames_salvos_1\frame_5.png
Salvo: frames_salvos_1\frame_6.png
Salvo: frames_salvos_1\frame_7.png
Salvo: frames_salvos_1\frame_8.png
Salvo: frames_salvos_1\frame_9.png
Salvo: frames_salvos_1\frame_10.png
Salvo: frames_salvos_1\frame_11.png
Salvo: frames_salvos_1\frame_12.png
Salvo: frames_salvos_1\frame_13.png
Salvo: frames_salvos_1\frame_14.png
Salvo: frames_salvos_1\frame_15.png
Salvo: frames_salvos_1\frame_16.png
Salvo: frames_salvos_1\frame_17.png
Salvo: frames_salvos_1\frame_18.png
Salvo: frames_salvos_1\frame_19.png
Salvo: frames_salvos_1\frame_20.png
Salvo: frames_salvos_1